# Big Data (UNT) -  2024

## Trabajo Práctico 2 

### Fecha de entrega:
Jueves 26 de septiembre a las 13:00 hs

### Modalidad de entrega
- Al finalizar el trabajo práctico deben hacer un último <i>commit</i> en su repositorio de GitHub llamado **“Entrega final del tp”**. 
- Asegurense de haber creado una carpeta llamada TP2. Deben entregar un reporte (pdf) y el código (jupyter notebook). Ambos deben estar dentro de esa carpeta.
- También deben enviar el link de su repositorio -para que pueda ser clonado y corregido- en el canal de Slack **#tp-entregas**.
- La última versión en el repositorio es la que será evaluada. Por lo que es importante que: 
    - No envien el mensaje en el canal de Slack **#tp-entregas** hasta no haber terminado y estar seguros de que han hecho el <i>commit y push</i> a la versión final que quieren entregar. Debido a que se pueden tomar hasta 3 días de extensión a lo largo del curso, no se corregirán sus tareas hasta no recibir el mensaje en el canal **#tp-entregas** y me tageen (@marianr2).
    - No hagan nuevos <i>push</i> después de haber entregado su versión final. Esto generaría confusión acerca de que versión es la que quieren que se les corrija. 

### Reglas de formato y presentation
- El trabajo debe tener una extensión máxima de 8 páginas (no se permite Apéndice). Se espera una buena redacción en la resolución del práctico.
- El informe debe ser entregado en formato PDF, con los gráficos e imágenes en este mismo archivo.
- Entregar el código con los comandos utilizados, identificando claramente a qué inciso corresponde cada comando.

### Parte I: Limpieza de la base

#### Ejercicio 1
La base de datos provista contiene información sobre oferentes de Airbnb en la ciudad de Nueva York. Realicen una limpieza de la base.
- Tengan en cuenta si hay valores duplicados y eliminénlos.
- Eliminen las columnas que no tienen información de interés.
- (Opcional) Luego de leer el artículo __[Missing-data imputation](https://bookdown.org/mike/data_analysis/imputation-missing-data.html##imputation-missing-data)__ (__[Missing-data imputation articulo alternativo](http://www.stat.columbia.edu/~gelman/arm/missing.pdf)__), decidan qué hacer con los missing values e implementen su decision. Justifiquen su eleccion. Pueden usar informacion del paper mencionado o de otras fuentes (citenlas si las usan) para explicar los problemas que pueden surgir por su estrategia elegida.
- Si hay observaciones con *outliers* o valores que no tienen sentido, tomen una decision. Expliquen las decisiones tomadas.
- Transformen las variables **‘neighbourhood_group’** y **‘room_type’** a variables numericas.
- Con la ayuda de los comandos <code>groupby</code> y <code>join</code> o <code>merge</code>, creen una columna que tenga la cantidad de oferentes por “Neighbourhood group”. Llamen a esa nueva columna <code>offer group</code>

In [1]:
#Instalamos las bibliotecas necesarias:
!pip install pandas openpyxl

In [3]:
import pandas as pd
#Cargamos el archivo de excel
df = pd.read_excel("C:/Users/piliv/OneDrive/Documentos/GitHub/-Big-Data-UNT--Grupo-1-/TP02/Base Limpia Airbnb NY.xlsx")
#El siguiente comando nos enseña las primeras filas de la base que cargamos
print(df.head())

     id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park   2787.0   
1  2595                             Skylit Midtown Castle   2845.0   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !   4632.0   
3  3831                   Cozy Entire Floor of Brownstone   4869.0   
4  5022  Entire Apt: Spacious Studio/Loft by central park   7192.0   

     host_name neighbourhood_group neighbourhood   latitude  longitude  \
0         John            Brooklyn    Kensington  4064749.0 -7397237.0   
1     Jennifer           Manhattan       Midtown  4075362.0 -7398377.0   
2    Elisabeth           Manhattan        Harlem  4080902.0  -739419.0   
3  LisaRoxanne            Brooklyn  Clinton Hill  4068514.0 -7395976.0   
4        Laura           Manhattan   East Harlem  4079851.0 -7394399.0   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room  149.0             1.0                9

In [7]:
#Llamamos nuevo_df a nuestro dataframe que ya no contiene los duplicados y verificamos que los eliminamos
nuevo_df = df_sin_duplicados
num_duplicados = nuevo_df.duplicated().sum()
print(f"Número de valores duplicados: {num_duplicados}")

Número de valores duplicados: 0


In [9]:
# Queremos observar el número de datos faltantes por columna. Usamos los comandos:
missing_values = nuevo_df.isnull().sum()
print(missing_values)

id                                  161
name                                 41
host_id                             285
host_name                           307
neighbourhood_group                 285
neighbourhood                       285
latitude                            285
longitude                           285
room_type                           285
price                               300
minimum_nights                      285
number_of_reviews                   285
last_review                       10323
reviews_per_month                 10323
calculated_host_listings_count      285
availability_365                    285
dtype: int64


In [11]:
# Eliminamos las 285 observaciones para las cuales tenemos datos faltantes en la variable host_id
nuevo_df = nuevo_df.dropna(subset=['host_id'])
missing_values = nuevo_df.isnull().sum()
print(missing_values)

id                                  160
name                                 26
host_id                               0
host_name                            22
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                15
minimum_nights                        0
number_of_reviews                     0
last_review                       10038
reviews_per_month                 10038
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [13]:
#Como las columnas id, host_id, host_name y name ya fueron usadas para eliminar duplicados, faltan de manera aleatoria
#y no se espera que tengan ninguna relevancia en el análisis y predicción de los datos, optamos por eliminarlas.
nuevo_df = nuevo_df.drop(columns=['id', 'name', 'host_id', 'host_name'])
missing_values = nuevo_df.isnull().sum()
print(missing_values)

neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                15
minimum_nights                        0
number_of_reviews                     0
last_review                       10038
reviews_per_month                 10038
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [15]:
#En el caso de los datos faltantes para la variable price decidimos imputar por la media CONDICIONAL al tipo de habitación
#y al vecindario al que pertenece el airbnb cuyo precio no fue reportado. Para ello usamos el comando:
datos_faltantes = nuevo_df.loc[nuevo_df['price'].isnull(), ['room_type', 'neighbourhood']]
print(datos_faltantes)

             room_type       neighbourhood
4506   Entire home/apt     Lower East Side
12567     Private room          Greenpoint
12925     Private room         South Slope
25430     Private room          Kensington
26821     Private room             Belmont
27093  Entire home/apt  University Heights
28238  Entire home/apt             Midtown
35308     Private room         Sunset Park
35641     Private room            Flushing
35679     Private room             Tribeca
37140  Entire home/apt         South Slope
41176     Private room        Williamsburg
42039  Entire home/apt         Murray Hill
44047     Private room           Ridgewood
48890     Private room            Bushwick


In [17]:
#Calculamos los precios medios condicionales a la combinación de room_type y neighbouhood correspondiente a las 
#observaciones que tienen datos faltantes de la variable price
precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Entire home/apt') & 
    (nuevo_df['neighbourhood'] == 'Lower East Side'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Greenpoint'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'South Slope'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Kensington'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Belmont'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Entire home/apt') & 
    (nuevo_df['neighbourhood'] == 'University Heights'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Entire home/apt') & 
    (nuevo_df['neighbourhood'] == 'Midtown'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Sunset Park'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Flushing'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Tribeca'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Entire home/apt') & 
    (nuevo_df['neighbourhood'] == 'South Slope'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Williamsburg'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Entire home/apt') & 
    (nuevo_df['neighbourhood'] == 'Murray Hill'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Ridgewood'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

precio_media = nuevo_df.loc[
    (nuevo_df['room_type'] == 'Private room') & 
    (nuevo_df['neighbourhood'] == 'Bushwick'),
    'price'
].mean()
print(f"La media del precio es: {precio_media}")

La media del precio es: 234.6158415841584
La media del precio es: 88.40631578947368
La media del precio es: 85.74725274725274
La media del precio es: 59.30927835051546
La media del precio es: 53.94444444444444
La media del precio es: 120.0
La media del precio es: 301.35122838944494
La media del precio es: 77.82843137254902
La media del precio es: 63.37581699346405
La media del precio es: 192.54545454545453
La media del precio es: 177.2139037433155
La media del precio es: 85.7565130260521
La media del precio es: 237.8391959798995
La media del precio es: 56.33922261484099
La media del precio es: 62.593804792518995


In [19]:
nuevo_df.at[4506, 'price'] = 234.6
nuevo_df.at[12567, 'price'] = 88.4
nuevo_df.at[12925, 'price'] =85.7
nuevo_df.at[25430, 'price'] = 59.3
nuevo_df.at[26821, 'price'] = 53.9

nuevo_df.at[27093, 'price'] = 120.0
nuevo_df.at[28238, 'price'] = 301.4
nuevo_df.at[35308, 'price'] = 77.8
nuevo_df.at[35641, 'price'] = 63.4
nuevo_df.at[35679, 'price'] = 192.5

nuevo_df.at[37140, 'price'] = 177.2
nuevo_df.at[41176, 'price'] = 85.8
nuevo_df.at[42039, 'price'] = 237.8
nuevo_df.at[44047, 'price'] = 56.3
nuevo_df.at[48890, 'price'] = 62.6

missing_values = nuevo_df.isnull().sum()
print(missing_values)

neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10038
reviews_per_month                 10038
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [21]:
#Transformo la variable neighbourhood_group
mapeo_neighbourhood_group = {
    'Bronx': 1,
    'Brooklyn': 2,
    'Manhattan': 3,
    'Queens': 4,
    'Staten Island': 5
}
nuevo_df['neighbourhood_group'] = nuevo_df['neighbourhood_group'].map(mapeo_neighbourhood_group)
frecuencias = nuevo_df['neighbourhood_group'].value_counts()
print(frecuencias)

neighbourhood_group
3    21635
2    20065
4     5627
1     1090
5      372
Name: count, dtype: int64


In [25]:
#Guardo los avances en un excel
df.to_excel('mi_dataframe.xlsx', index=False)

In [27]:
frecuencia = df['room_type'].value_counts()
print(frecuencia)

room_type
Entire home/apt    25371
Private room       22283
Shared room         1145
Name: count, dtype: int64


In [29]:
#Transformo la variable room_type
mapeo_room_type = {
    'Entire home/apt': 1,
    'Private room': 2,
    'Shared room': 3
}
nuevo_df['room_type'] = nuevo_df['room_type'].map(mapeo_room_type)
frecuencias = nuevo_df['room_type'].value_counts()
print(frecuencias)

room_type
1    25366
2    22278
3     1145
Name: count, dtype: int64


### Parte II: Graficos y visualizaciones

#### Ejercicio 2
Una vez hecha la limpieza, realicen una matriz de correlación con las siguientes variables:**‘neighbourhood group’, ‘latitude’, ‘longitude’, ‘room type’,‘price’, ‘minimum nights’, ‘number of reviews’, ‘reviews per month’,‘calculated host listings count’, ‘availability 365’.** Comenten los resultados. Utilicen alguno de los comandos disponibles en este link para graficar la matriz de correlacion.

*Nota:* consideren cómo es conveniente incluir las variables que originalmente eran categóricas para poder interpretar mejor la matriz de correlación.

In [ ]:
# Resolver acá


#### Ejercicio 3
Respondan las siguientes preguntas: ¿Cuál es la proporción de oferentes por “Neighbourhood group”? ¿Y por tipo de habitación? Además, realicen gráficos para mostrar estas composiciones y comenten los resultados.

In [ ]:
# Resolver acá


#### Ejercicio 4
Realicen un histograma de los precios de los alojamientos. Comenten el gráfico obtenido. Además, respondan las siguientes preguntas: ¿cuál es el precio mínimo, máximo y promedio? ¿Cuál es la media de precio por “Neighbourhood group” y por tipo de habitación? 

In [ ]:
# Resolver acá


#### Ejercicio 5
Realicen dos scatter plots con dos variables de interés en cada uno. Comenten.

In [1]:
# Resolver acá


#### Ejercicio 6
Utilicen el análisis de componentes principales para graficar las variables en dos dimensiones. Comenten los resultados obtenidos (qué porcentaje de la varianza se logra explicar con dos componentes, cómo son los *loadings*, si ven algún patrón en el gráfico).

In [1]:
# Resolver acá


### Parte III: Predicción

El objetivo de esta parte del trabajo es intentar predecir los precios de los alojamientos.

#### Ejercicio 7
Eliminen de la base todas las variables relacionadas al precio

In [ ]:
# Resolver acá


#### Ejercicio 8
Partan la base en una base de prueba (test) y una de entrenamiento (train) utilizando el comando <code>train_test_split</code>. La base de entrenamiento debe comprender el 70% de los datos, y la semilla a utilizar (*random state instance*) debe ser 201. Establezca a price como su variable dependiente en la base de entrenamiento (vector y). El resto de las variables ser´an las variables independientes (matriz X). Recuerden agregar la columna de unos (1).

In [ ]:
# Resolver acá


#### Ejercicio 9
Implementen una regresión lineal y comenten los resultados obtenidos.

In [ ]:
# Resolver acá
